## PANDAS DATA MANIPULATION QUICK RECIPES
John Steedman, 27/09/2022

## Contents  <a class="anchor" id="Home"></a>


#### Get Data
* [Start Here](#SS1)


#### Simple Operations

* [Number of rows in the dataset](#A1)
* [First 100 Rows](#A2)
* [Set Data Types](#A3)
* [Create Date from String](#A4)
* [Get Unique Values (one column)](#A5)
* [Get Unique Values (multiple columns)](#A6)
* [Add a new column](#A7)

#### Selecting, Filtering and Updating

* [Filter rows based on one or more conditions (one condition)](#B1)
* [Filter rows based on one or more conditions (multiple conditions)](#B2)
* [Filter rows negating conditions](#B3)
* [Filter rows specifying one condition or the other](#B4)
* [Updating based on a selection](#B5)
* [Reorder Columns](#B7)
* [Delete Columns](#B8)
* [Rename Columns](#B9)
* [Filter based on String ( wildcard )](#B10)


#### Sorting

* [Sorting based on one column](#C1)
* [Sorting based on multiple columns](#C2)


#### Aggregate/Summarise

* [Aggregate over one or more  columns using a variety of functions (sum,mean etc)](#D1)
* [Quick counts across multiple combinations - simpler version of above but very useful (table in R)](#D2)
* [Find all distinct values across a range of columns (make a dimension table)](#D3)
* [Sum over case statements returning 0 or 1](#D4)
* [Contingency Table](#D5)


#### Derived Columns ( Categorization)

* [Case Statement](#E1)
* [Nested Case Statement](#E2)
* [Band numeric values to categories](#E3)
* [TODO Group up infrequent or low impact "Other" Values](#E4)



#### Derived Columns 2 (Window Functions)

* [Count Within Group](#F1)


#### Derived Columns 3 (Dates)

* [Obtain First Occurrence of Date Within Group](#G1)
* [Days till Event](#G2)
* [Date add](#G3)
* [Extract Year/Month from Date](#G4)


####  Derived Columns 4 (More Windows and Functions)

* [Add a partition based ranking, e.g. sales rank within a category](#H1)
* [Add a partition based ranking, e.g. sales rank within two category columns](#H2)
* [Bring down values from a previous row, e.g. the previous transaction date within accountID group](#H3)
* [Update a value based on a function that takes in two existing columns](#H4)
* [Add two column values together](#H5)
* [Update a slice of values based on an index sl](#H6)

####  Derived Columns 5 (String Functions)

* [Split strings](#I1)
* [Extract a substring](#I2)
* [Trim a String](#I3)

#### Reshaping Data

* [Pivot](#J1)
* [Unpivot](#J2)
* [Pivoting 2+ columns values](#J3)

#### Linking Two Data Tables

* [Left Join ](#K1)
* [Union](#K2)

#### Data Cleaning - Null and Blank Values
Nulls are ignored in pandas even in counting rows.

* [Identify and Replace Nulls](#L1)
* [Identify and Replace Blank Strings](#L2)
* [TODO: Remove White Space Invisible Characters](#L2)

#### Data Cleaning - Duplicate Values

* [Flag duplicates across a set of columns](#M1)
* [Drop duplicates across a set of columns](#M2)

#### Data Profiling - 

* [TODO: Summary statistics for a numerical value](#O1)
* [TODO: Counts for a categorical Value](#O2)

#### Other/Various

* [TODO: Historic snapshots from transactions](#O1)
* [TODO: Iterate through data frame](#O2)
* [TODO: TopN across groups](#O2)



## Obtain Sample Dataset <a class="anchor" id="SS1"></a>

In [3]:
import numpy as np
import pandas as pd
import requests

#https://github.com/microsoft/DataStoriesSamples/tree/master/samples/FraudDetectionOnADL
data_url = "https://raw.githubusercontent.com/microsoft/DataStoriesSamples/master/samples/FraudDetectionOnADL/Data/transactions.csv"
df = pd.read_csv ( data_url )


In [4]:
cols = [  'accountID'             , 'transactionID'
        , 'transactionAmountUSD'  ,'transactionCurrencyCode' 
        , 'localHour'             , 'transactionScenario' 
        , 'transactionType'       , 'transactionIPaddress' 
        , 'ipState'               , 'ipPostalCode' 
        , 'ipCountry'             , 'isProxyIP' 
        , 'browserLanguage'       , 'paymentInstrumentType' 
        , 'cardType' 
        , 'paymentBillingPostalCode' 
        , 'paymentBillingState' 
        , 'paymentBillingCountryCode' 
        , 'shippingPostalCode' 
        , 'shippingState' 
        ,  'shippingCountry' 
        , 'cvvVerifyResult' 
        , 'digitalItemCount'
        , 'physicalItemCount'
        , 'transactionDateTime' ]

In [5]:
df.columns = cols

In [6]:
# This is a 1/2 milllion rows so just filter to the first 20,000 while practicing
# (later ignore this step to check processing speed)
df = df.iloc [ 0:20000,  ]
df.shape

(20000, 25)

[Back to contents](#Home)

### Simple Operations

#### Number of Rows in Dataset  <a class="anchor" id="A1"></a>

In [7]:
len(df.index)
# Or the following
df.shape[0]
# This below will ignore NA in it's counting so use with care.
df[df.columns[0]].count()

20000

[Back to contents](#Home)

#### First 100 Rows<a class="anchor" id="A2"></a>

In [8]:
df_res = df.iloc [0:100,]
df_res.shape

(100, 25)

[Back to contents](#Home)

#### Set data types <a class="anchor" id="A3"></a>

In [9]:
df.transactionCurrencyCode = df.transactionCurrencyCode.astype ('string')
df.paymentBillingCountryCode = df.paymentBillingCountryCode.astype ('string')
df.transactionCurrencyCode.dtype

string[python]

[Back to contents](#Home)

#### Create date from String <a class="anchor" id="A4"></a>

In [10]:
#https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
df['transactionDateTime_str'] = df['transactionDateTime'].astype ('string')

# 5/16/2007 11:37
# Split the date time into just the date part

df['transactionDate_str'] = df['transactionDateTime_str'].str.split (expand=True).loc [ : , 0 ]
df ['transactionDate']    = pd.to_datetime(df['transactionDate_str'] , format='%m/%d/%Y', errors='coerce' ) 


[Back to contents](#Home)

#### Get Unique Values (One Column) <a class="anchor" id="A5"></a>

In [11]:
df.transactionCurrencyCode.unique()  [0: 9+1]

<StringArray>
['USD', 'BRL', 'AUD', 'EUR', 'SEK', 'DKK', 'SGD', 'GBP', 'NOK', 'MXN']
Length: 10, dtype: string

In [12]:
# Or why not count hem 
df.transactionCurrencyCode.value_counts () [0:9+1]

USD    13850
EUR     1548
GBP     1146
CAD      925
AUD      572
BRL      266
JPY      177
NOK      172
SEK      158
DKK      152
Name: transactionCurrencyCode, dtype: Int64

[Back to contents](#Home)

#### Get Unique Values (Multiple Columns) <a class="anchor" id="A6"></a>

In [13]:
req_cols = ['transactionCurrencyCode', 'paymentBillingCountryCode' ]
df_sel = df [req_cols]
#df_sel.drop_duplicates().head()
# or with a new index
df_sel.drop_duplicates().reset_index(drop=True).tail(3) # 662 rows

,transactionCurrencyCode,paymentBillingCountryCode
660,KES,US
661,MYR,GB
662,CZK,CH


[Back to contents](#Home)

#### Add a new column <a class="anchor" id="A7"></a>

In [890]:
df['BILLED_IN_US'] = 'No'

[Back to contents](#Home)

# Selection, Filtering and Updating

#### Filter rows based on one or more conditions (one condition)<a class="anchor" id="B1"></a>

In [891]:
# Method 1 : boolean indexing, note that numpy will throw an error is there are nas
df_res = df [ df['paymentBillingCountryCode'].fillna('') == 'US' ]
df_res.paymentBillingCountryCode.unique() [0]

'US'

In [892]:
# Method 2: 
# Is there a better way?

[Back to contents](#Home)

#### Filter rows based on one or more conditions (multiple conditions)<a class="anchor" id="B2"></a>

In [893]:
# Method: booleanIndexindexing, note the brackets
boolIndex = ( df['transactionCurrencyCode'].fillna('') == 'USD' ) & \
            ( df['paymentBillingState'].fillna('') == 'AZ'  )

# or, whichi is probably better:
boolIndex = \
np.logical_and ( df['transactionCurrencyCode'].fillna('') == 'USD' , \
                 df['paymentBillingState'].fillna('') == 'AZ' )

df_res = df [ boolIndex ]
df_res [['transactionCurrencyCode', 'paymentBillingState']].reset_index(drop=True).drop_duplicates()
df_res [['transactionCurrencyCode', 'paymentBillingState']].count()

transactionCurrencyCode    175
paymentBillingState        175
dtype: int64

[Back to contents](#Home)

####  Filter rows negating conditions<a class="anchor" id="B3"></a>

In [894]:
boolIndex = ~(( df['transactionCurrencyCode'].fillna('') == 'USD' ) & \
              ( df['paymentBillingState'].fillna('') == 'AZ'  ))

df_res = df [ boolIndex ]
df_res [['transactionCurrencyCode', 'paymentBillingState']].count() + 175

transactionCurrencyCode    20000
paymentBillingState        14289
dtype: int64

[Back to contents](#Home)

####  Filter rows specifying one condition or the other<a class="anchor" id="B4"></a>

In [895]:
boolIndex = (  df['transactionCurrencyCode'].fillna('') == 'USD' ) | \
            ( df['paymentBillingState'].fillna('') == 'AZ'  )

df_res = df [ boolIndex ]
df_res [['transactionCurrencyCode', 'paymentBillingState']].count() 

transactionCurrencyCode    13922
paymentBillingState         9947
dtype: int64

[Back to contents](#Home)

#### Updating based on a selection<a class="anchor" id="B5"></a>

In [896]:
boolIndex = (  df['paymentBillingCountryCode'].fillna('') == 'US' )
df.loc [   boolIndex  ,'BILLED_IN_US'  ] = 'Yes'
df [~boolIndex] ['paymentBillingCountryCode'].value_counts() [0: 5-1]

GB    1307
CA     888
AU     681
DE     488
Name: paymentBillingCountryCode, dtype: Int64

[Back to contents](#Home)

#### Select a set of columns<a class="anchor" id="B6"></a>

In [55]:
req_cols = [ 'paymentBillingCountryCode', 'transactionCurrencyCode' ]
df [req_cols].head(2)

,paymentBillingCountryCode,transactionCurrencyCode
0,GB,USD
1,US,USD


[Back to contents](#Home)

#### Reorder Columns  - as above.<a class="anchor" id="B7"></a>

#### Delete column(s)<a class="anchor" id="B8"></a>

In [36]:
df_temp = df[0:10]
# note the need for axis = 1 (argh)
'accountID' in df_temp.drop ([  'accountID', 'transactionCurrencyCode'] , axis=1).columns

False

[Back to contents](#Home)

#### Rename Columns <a class="anchor" id="B9"></a>

In [899]:
df_temp = df[0:10]
df_temp.rename(columns={  'accountID': 'account_id'  
                        , 'transactionID': 'transaction_eye_d' }) \
              .columns

Index(['account_id', 'transaction_eye_d', 'transactionAmountUSD',
       'transactionCurrencyCode', 'localHour', 'transactionScenario',
       'transactionType', 'transactionIPaddress', 'ipState', 'ipPostalCode',
       'ipCountry', 'isProxyIP', 'browserLanguage', 'paymentInstrumentType',
       'cardType', 'paymentBillingPostalCode', 'paymentBillingState',
       'paymentBillingCountryCode', 'shippingPostalCode', 'shippingState',
       'shippingCountry', 'cvvVerifyResult', 'digitalItemCount',
       'physicalItemCount', 'transactionDateTime', 'transactionDateTime_str',
       'transactionDate_str', 'transactionDate', 'BILLED_IN_US'],
      dtype='object')

[Back to contents](#Home)

#### Filter based on String ( wildcard )<a class="anchor" id="B10"></a>

In [900]:
# page 212 of 
df [  df.ipState.fillna('').str.contains ('south') ] ['ipState'] [0:4+1]

26     south carolina
38    new south wales
47       south dakota
54    new south wales
55    new south wales
Name: ipState, dtype: object

[Back to contents](#Home)

## Ordering

[Back to contents](#Home)

#### Ordering based on one column<a class="anchor" id="C1"></a>

In [901]:
df_res = df.sort_values ( ['ipState'] , ascending = False ) 
df_res['ipState'] 

7604       zurich
9282       zurich
1245       zurich
18089      zurich
3256       zurich
           ...   
13342    a coruna
1666          NaN
3338          NaN
5413          NaN
10823         NaN
Name: ipState, Length: 20000, dtype: object

[Back to contents](#Home)

#### Ordering based on multiple columns<a class="anchor" id="C2"></a>

In [902]:
df_res = df.sort_values (  ['ipState', 'accountID' ], ascending=[False, True])

df_res [ [ 'ipState', 'accountID'  ]].reset_index(drop=True) [0:9+1].head(5) 

,ipState,accountID
0,zurich,A5018736024291620
1,zurich,A8217517060477503
2,zurich,C2318401957742700
3,zurich,D8184933477543700
4,zurich,I5824243965868831


[Back to contents](#Home)

# Aggregate / Summarise

.Aggregate over one or more  columns using a variety of functions (sum,mean etc)
.Quick counts across multiple combinations - simpler version of above but very useful (table in R)
.Find all distinct values across a range of columns (make a dimension table)


#### Aggregate over one or more  columns using a variety of functions (sum,mean etc)<a class="anchor" id="D1"></a>

In [1001]:
#https://stackoverflow.com/questions/12589481/multiple-aggregations-of-the-same-column-using-pandas-groupby-agg
dfRes = df.groupby('ipCountry').agg(
      transactionAmountUSD_sum=('transactionAmountUSD', 'sum')
    , transactionAmountUSD_count=('transactionAmountUSD', 'count') ) \
    .sort_values('transactionAmountUSD_sum', ascending= [False] )

dfRes[['transactionAmountUSD_sum']] [0: 2+1 ]


,transactionAmountUSD_sum
ipCountry,
us,12988907.20
gb,1197224.58
ca,858710.14


[Back to contents](#Home)

#### Quick counts across multiple combinations - simpler version of above but very useful (table in R)<a class="anchor" id="D2"></a>

In [904]:
df.groupby ('ipCountry')[['accountID', 'transactionAmountUSD']].count ()[0:3+1]
# This is still a bit slower than table() in R

,accountID,transactionAmountUSD
ipCountry,,
ad,1,1
ae,21,21
af,12,12
al,1,1


[Back to contents](#Home)

#### Find all distinct values across a range of columns (make a dimension table)<a class="anchor" id="D3"></a>

In [905]:
# This was done above but repeating
df[ ['shippingCountry', 'transactionCurrencyCode', 'ipState'] ].drop_duplicates().reset_index(drop=True).tail(6) # 662 rows

,shippingCountry,transactionCurrencyCode,ipState
2964,GB,MYR,bristol
2965,CZ,USD,british columbia
2966,JP,USD,niigata
2967,FR,GBP,herault
2968,CH,CZK,zuerich
2969,NaN,USD,guayas


[Back to contents](#Home)

#### Sum over case statements returning 0 or 1<a class="anchor" id="D4"></a>
This is a way in SQL build up lots of summary columns of the type: number_of_x, number_of_y
<br> This seems quite long-winded


In [906]:
# check for nulls as ever
#df['localHour'].isna().value_counts()
#df['cardType'].isna().value_counts() # has some nulls

# Probably has to be done with temporary columns
df['cond_office_hours'] = np.where ( np.logical_and (df.localHour <= 17, df.localHour >=8 ) , 1,0  )
df['payment_VISA_AMEX'] = np.where ( df.cardType.isin ( ['VISA', 'AMEX'] ), 1,0)


df.groupby ('shippingCountry' ).agg (
     num_in_office_hrs=('cond_office_hours', 'sum' )
    ,num_with_VISA_AMEX =('payment_VISA_AMEX' ,'sum')
    ,total =('accountID' ,'count')
).head()

#Then it's necessary to drop the temp columns...

,num_in_office_hrs,num_with_VISA_AMEX,total
shippingCountry,,,
AD,1,1,2
AE,7,7,13
AF,1,3,3
AL,0,1,2
AN,1,1,1


[Back to contents](#Home)

#### Contingency Table<a class="anchor" id="D5"></a>

[See Pivot Table](#J1)

[Back to contents](#Home)

# Derive Columns 1 (categorization)

.Categorization : Case statement.
.Categorization : Nested Case Statement (?)
.Categorization : band numeric values to categories

#### "Case Statement"<a class="anchor" id="E1"></a>

In [907]:
#https://stackoverflow.com/questions/49228596/pandas-case-when-default-in-pandas

df['derived_col1']=   np.where ( df ['ipState']=='bristol' , 'Bris' , 
                       np.where ( df ['ipState'].str.contains ('south'),
                         #Else         
                         'Southern State','Other'))
                             
df['derived_col1'].value_counts()

Other             19504
Southern State      488
Bris                  8
Name: derived_col1, dtype: int64

In [908]:
# Another Way. 

df['derived_col1_b'] = np.select ( 
    [
     df ['ipState']=='bristol' 
    ,df ['ipState']=='niigata'
    ,df ['transactionAmountUSD'].between (0,1)
    ]
    ,
    [
      'Bris'   
     ,'Niigata'   
     ,'$1-2 '

    ]
    ,  default=  'Other'
        
)

df['derived_col1_b'].value_counts()

Other      19977
$1-2          13
Bris           8
Niigata        2
Name: derived_col1_b, dtype: int64

[Back to contents](#Home)

#### Nested Case Statement<a class="anchor" id="E2"></a>

In [162]:
# Don't think this is directly possible

#### Band numeric values to categories<a class="anchor" id="E3"></a>

In [909]:
# with pd.cut the bins boundaries are inclusive on the low side.
# bin [0,5,7] will assign:
# 0-5 => (0,5) first bin range
# 6-7 => (5,7) second bin range

bins     = [ 0, 100, 500, 800, 1000,2000 ]
labels   =  ['0-100', '100.01-500', '500.01-800', '800.01-1000', '1000.01-2000']

df['Amount_Banded']  =   pd.cut ( df.transactionAmountUSD, bins=bins, labels = labels)

df.groupby('Amount_Banded').agg(
      min_Val=('transactionAmountUSD', 'min')
    , max_Val=('transactionAmountUSD', 'max')  
    , num_rows= ( 'transactionAmountUSD', 'count')
) \
.sort_values('Amount_Banded' )#, ascending= [False] )



,min_Val,max_Val,num_rows
Amount_Banded,,,
0-100,0.05,99.93,971
100.01-500,100.09,500.00,4098
500.01-800,500.04,799.94,3031
800.01-1000,800.06,999.77,1946
1000.01-2000,1000.05,1999.98,9954


In [718]:
# Check for blanks
df['Amount_Banded'].isna().value_counts()

False    20000
Name: Amount_Banded, dtype: int64

[Back to contents](#Home)

# Derive Columns 2 (Window Functions)

#### Count Within Group<a class="anchor" id="F1"></a>
<br>
E.g. if a customer has transactions, we'd like to know how many transactions there are overall


In [910]:
# First ensure that there are no nulls: this is important!
#df.accountID.isna().value_counts()

# Number of unique accountID values: 18154
#df.accountID.value_counts().reset_index ()['index'].nunique()

df['ones'] = 1
df.accountID = df.accountID.astype('string')

# get a 18154 row tables of first transactoin dates dates
df_agg = df.groupby('accountID')['ones'].count().reset_index().rename ( columns={'ones':'num_transactions'} )

if not 'num_transactions' in df.columns:
    df = df.merge (df_agg, how="left", on =["accountID"] )

df[['accountID', 'num_transactions', 'transactionAmountUSD']]\
.sort_values (['num_transactions', 'accountID'], ascending = [False,False]).head(7)


,accountID,num_transactions,transactionAmountUSD
1041,X2821285840602153,4,450.36
3953,X2821285840602153,4,1153.82
6891,X2821285840602153,4,1522.96
8323,X2821285840602153,4,1910.21
9063,W5713589171547099,4,453.37
11016,W5713589171547099,4,763.89
13330,W5713589171547099,4,860.92


[Back to contents](#Home)

# Derive Columns 3 (Dates) 

#### Obtain First Occurrence of Date Within Group<a class="anchor" id="G1"></a>
<br>
This is more easily done with Window Functions in SQL
<br>
Note that you must sort the data frame first.
<br>
This can be used to get the second occurrence etc as well

In [911]:
# First ensure that there are no nulls: this is important!
#df.accountID.isna().value_counts()
#df.transactionDate.isna().value_counts()

# Number of unique accountID values: 18154
#df.accountID.value_counts().reset_index ()['index'].nunique()

# get a 18154 row tables of first transaction dates dates
df_agg = df.sort_values('transactionDate', ascending=True).\
                       groupby('accountID').\
                       nth(0)['transactionDate'].\
                       reset_index ().\
                       rename ( columns={'transactionDate': 'transactionDate_first'})

if  'transactionDate_first' in df.columns:
    df.drop (columns = ['transactionDate_first'])
    
df = df.merge (df_agg, how="left", on ="accountID" )

cols =     ['accountID', 'transactionDate_first','transactionDate','num_transactions']

df[cols] \
.sort_values (['num_transactions', 'accountID'], ascending = [False,False])\
.head(5)


,accountID,transactionDate_first,transactionDate,num_transactions
1041,X2821285840602153,2007-02-11,2007-04-26,4
3953,X2821285840602153,2007-02-11,2007-06-24,4
6891,X2821285840602153,2007-02-11,2007-02-11,4
8323,X2821285840602153,2007-02-11,2007-11-01,4
9063,W5713589171547099,2007-02-10,2007-02-22,4


[Back to contents](#Home)

#### Days till Event<a class="anchor" id="G2"></a>

In [913]:
df['days_since_first_trans'] =   (df['transactionDate'] - df['transactionDate_first'])  / np.timedelta64 (1, 'D') 

#boolIndex = np.logical_and (df.num_transactions > 2 , df.days_since_first_trans > 0 )
boolIndex =  (df.num_transactions > 2 ) & (df.days_since_first_trans > 0 )

cols = ['accountID', 'transactionDate_first', 'transactionDate', 'days_since_first_trans']
df[boolIndex][cols].sort_values ( 'days_since_first_trans' )


,accountID,transactionDate_first,transactionDate,days_since_first_trans
14948,Y8216400785437216,2007-02-10,2007-02-11,1.0
726,L4721393138775624,2007-02-14,2007-02-15,1.0
17590,L4721393138775624,2007-02-14,2007-02-15,1.0
12787,R9629221560269986,2007-02-19,2007-02-21,2.0
12704,J1903878531994708,2007-02-10,2007-02-13,3.0
...,...,...,...,...
8061,R9453270213536424,2007-03-13,2007-12-26,288.0
14610,J8124281182721525,2007-02-19,2007-12-07,291.0
19216,N5192231467742956,2007-04-17,2008-02-06,295.0
9276,E4892731116568663,2007-02-11,2007-12-20,312.0


[Back to contents](#Home)

#### Date add<a class="anchor" id="G3"></a>

In [914]:

days_to_add = 10
df['transactionDate_plus_10d'] =   df['transactionDate']  + pd.Timedelta(days=days_to_add)
    
cols = ['accountID', 'transactionDate', 'transactionDate_first', 'days_since_first_trans','transactionDate_plus_10d']
df[cols].sort_values ( 'days_since_first_trans' ) 


,accountID,transactionDate,transactionDate_first,days_since_first_trans,transactionDate_plus_10d
9999,J7848752080701268,2007-05-31,2007-05-31,0.0,2007-06-10
13024,O5220992097989057,2007-09-18,2007-09-18,0.0,2007-09-28
13023,X7544532156724225,2007-07-10,2007-07-10,0.0,2007-07-20
13022,R5636403936032189,2007-05-01,2007-05-01,0.0,2007-05-11
13021,P4946688286855991,2007-03-08,2007-03-08,0.0,2007-03-18
...,...,...,...,...,...
19169,X6634962469904029,2008-02-06,2007-03-03,340.0,2008-02-16
5018,W8586167679002914,2008-01-25,2007-02-14,345.0,2008-02-04
10972,U2641852470896979,2008-02-05,2007-02-24,346.0,2008-02-15
91,C9414425912542449,2008-01-24,2007-02-10,348.0,2008-02-03


[Back to contents](#Home)

#### Extract Year and Month from Date<a class="anchor" id="G4"></a>

[Dependency](#A4)

In [16]:
df['trans_year'] = df.transactionDate.dt.year
df['trans_year'].value_counts()

# There are lots of functions to use
df['trans_month'] = df.transactionDate.dt.month
df['trans_month'] = df.transactionDate.dt.strftime("%b")

df['trans_month'].value_counts() [0:5]

Feb    4609
Mar    3879
Apr    2614
May    2179
Jun    1661
Name: trans_month, dtype: int64

[Back to contents](#Home)

## Derived Columns 4 ("Window Functions" type manipulation )

Derive columns 2

.Add a partition based ranking, e.g. sales rank within a category
.Bring down values from a previous row, e.g. the previous transaction date
.Creating a new column by running a function over one or more existing columns.
.Write a value to a column within a certain selection


#### Add a partition based ranking, e.g. sales rank within a category<a class="anchor" id="H1"></a>
<br>
method 1 : use groupby

In [915]:
# Create a ranking column
# Change method to 'dense as required'
df['rank_sales_by_browser']  = df.groupby('browserLanguage')['transactionAmountUSD']\
                                  .rank(method='dense', ascending=False)

# Show the 
df.sort_values ( ['browserLanguage', 'transactionAmountUSD'], ascending = [True, False]) \
            .loc [ :, ['browserLanguage', 'transactionAmountUSD',  'rank_sales_by_browser'] ] \
            .head(6)


,browserLanguage,transactionAmountUSD,rank_sales_by_browser
13997,ar-AE,1926.21,1.0
1079,ar-AE,1905.55,2.0
12206,ar-AE,1811.98,3.0
10487,ar-AE,1777.42,4.0
3738,ar-AE,1665.30,5.0
16574,ar-AE,1286.02,6.0


In [916]:
# Check the results within groups, this is a bit slow
# TODO: find a better way to do this

df.sort_values (['browserLanguage','rank_sales_by_browser']).groupby('browserLanguage') \
.head(2).reset_index(drop=True) \
.loc[:,  ['browserLanguage','transactionAmountUSD','rank_sales_by_browser']  ] \
.head(9)


,browserLanguage,transactionAmountUSD,rank_sales_by_browser
0,ar-AE,1926.21,1.0
1,ar-AE,1905.55,2.0
2,ar-DZ,1080.10,1.0
3,ar-EG,1623.67,1.0
4,ar-KW,1868.20,1.0
5,ar-KW,1333.82,2.0
6,ar-LB,1112.57,1.0
7,ar-OM,1784.93,1.0
8,ar-OM,1553.49,2.0


[Back to contents](#Home)

Method 2: By resetting the index, not sure if this is quicker.

In [917]:
df_temp = df.set_index ( 'browserLanguage')
df_temp['rank_sales_by_browser2']  = df_temp['transactionAmountUSD'].groupby(level=0).rank(  method='dense', ascending=False)

df_temp = df_temp.reset_index ()

# Compare the results with the previous method in above cells.

df_temp.sort_values ( ['browserLanguage', 'transactionAmountUSD'], ascending = [True, False]) \
            .loc [ :, ['browserLanguage', 'transactionAmountUSD',  'rank_sales_by_browser', 'rank_sales_by_browser2'] ] \
            .head(6)


,browserLanguage,transactionAmountUSD,rank_sales_by_browser,rank_sales_by_browser2
13997,ar-AE,1926.21,1.0,1.0
1079,ar-AE,1905.55,2.0,2.0
12206,ar-AE,1811.98,3.0,3.0
10487,ar-AE,1777.42,4.0,4.0
3738,ar-AE,1665.30,5.0,5.0
16574,ar-AE,1286.02,6.0,6.0


[Back to contents](#Home)

#### Add a partition based ranking, e.g. sales rank within two category columns<a class="anchor" id="H2"></a>

In [918]:
# Create a ranking column
# Change method to 'dense as required'

df['rank_sales_by_ip_country']  = df.groupby(['shippingCountry', 'ipCountry'])['transactionAmountUSD'] \
.rank(method='dense', ascending=False)

df.sort_values ( ['isProxyIP', 'ipCountry' ,'transactionAmountUSD'], ascending = [False, False, False]) \
    .loc [ :,  ['isProxyIP', 'ipCountry', 'transactionAmountUSD', 'rank_sales_by_ip_country' ] ] \
    .head(5)


,isProxyIP,ipCountry,transactionAmountUSD,rank_sales_by_ip_country
12337,True,zm,1451.16,1.0
19747,True,za,1751.23,2.0
19046,True,za,530.49,23.0
12802,True,za,347.61,26.0
2879,True,uy,307.39,2.0


[Back to contents](#Home)

#### Bring down values from a previous row, e.g. the previous transaction date within accountID group<a class="anchor" id="H3"></a>


In [919]:
df ['transactionDate_prev']  = df.sort_values('transactionDate').groupby('accountID')['transactionDate'].shift (1)
#df.groupby('accountID')['transactionDate'].shift ()

boolIndex = df.num_transactions == 3 
df.sort_values ( ['accountID', 'transactionDate'])\
     .loc [boolIndex, [ 'accountID', 'num_transactions' , 'transactionDate_prev','transactionDate' ]]


,accountID,num_transactions,transactionDate_prev,transactionDate
19909,A1566132399521989,3,NaT,2007-03-02
11121,A1566132399521989,3,2007-03-02,2007-03-08
8903,A1566132399521989,3,2007-03-08,2007-03-09
4675,A2760793813048785,3,2007-04-22,2007-04-22
13359,A2760793813048785,3,NaT,2007-04-22
...,...,...,...,...
8038,Z6708428249935724,3,2007-02-10,2007-04-11
7031,Z6708428249935724,3,2007-04-11,2007-06-01
16707,Z9959815974415865,3,NaT,2007-02-11
9140,Z9959815974415865,3,2007-02-11,2007-06-06


In [920]:
df['days_since_prev_trans'] = -1 * (df ['transactionDate'] - df ['transactionDate_prev'] ) /  np.timedelta64 (1, 'D') 
df.loc [ df['days_since_prev_trans'].isna(), 'days_since_prev_trans'] = 0 

df.sort_values ( ['accountID', 'transactionDate'])\
     .loc [boolIndex, [ 'accountID', 'num_transactions' , 'transactionDate', 
                       'transactionDate_prev', 'days_since_prev_trans']]


,accountID,num_transactions,transactionDate,transactionDate_prev,days_since_prev_trans
19909,A1566132399521989,3,2007-03-02,NaT,0.0
11121,A1566132399521989,3,2007-03-08,2007-03-02,-6.0
8903,A1566132399521989,3,2007-03-09,2007-03-08,-1.0
4675,A2760793813048785,3,2007-04-22,2007-04-22,0.0
13359,A2760793813048785,3,2007-04-22,NaT,0.0
...,...,...,...,...,...
8038,Z6708428249935724,3,2007-04-11,2007-02-10,-60.0
7031,Z6708428249935724,3,2007-06-01,2007-04-11,-51.0
16707,Z9959815974415865,3,2007-02-11,NaT,0.0
9140,Z9959815974415865,3,2007-06-06,2007-02-11,-115.0


[Back to contents](#Home)

#### Update a value based on a function that takes in two existing columns<a class="anchor" id="H4"></a>
<br>
This is not vectorized and is very slow.

In [921]:
def calculate_days_between(currDate,prevDate):
    if pd.isnull (prevDate) :
        prevDate= currDate
    return  -1 * ( (currDate- prevDate)  /  np.timedelta64 (1, 'D') )

df['days_since_prev_trans2'] = df.apply(lambda x: calculate_days_between(x.transactionDate, x.transactionDate_prev), axis=1)

df[df.num_transactions >= 2].sort_values ( ['accountID', 'transactionDate'])\
     .loc [:, [ 'accountID', 'num_transactions' , 'transactionDate', 'transactionDate_prev'
               , 'days_since_prev_trans', 'days_since_prev_trans2']].head(7)


,accountID,num_transactions,transactionDate,transactionDate_prev,days_since_prev_trans,days_since_prev_trans2
18167,A1032246091865881,2,2007-05-14,NaT,0.0,-0.0
8962,A1032246091865881,2,2007-06-18,2007-05-14,-35.0,-35.0
1857,A1049243531493622,2,2007-05-02,NaT,0.0,-0.0
8247,A1049243531493622,2,2007-06-23,2007-05-02,-52.0,-52.0
8366,A1431743378107759,2,2007-02-21,NaT,0.0,-0.0
3388,A1431743378107759,2,2007-03-09,2007-02-21,-16.0,-16.0
3918,A1462374232505055,2,2007-07-31,NaT,0.0,-0.0


[Back to contents](#Home)

#### Add two column values together<a class="anchor" id="H5"></a>
<br>


In [922]:
df['ones'] = 1
df['twos'] = 2

df['result'] = df['ones'] + df['twos']
cols = ['ones', 'twos', 'result']
df[cols].head()

,ones,twos,result
0,1,2,3
1,1,2,3
2,1,2,3
3,1,2,3
4,1,2,3


[Back to contents](#Home)

#### Update a slice of values based on an index slice<a class="anchor" id="H6"></a>
<br>
This is just a matter of logic, nothing new here.  Create a boolean slicer and then just do an update within that index slice.

In [923]:
df['num_transactions_banded'] = 'one'
booleanIndex = df.num_transactions > 1

# This is a good feature of ordered indexing which holds it's order
df.loc [ booleanIndex, 'num_transactions_banded' ] =  df.loc[booleanIndex, 'num_transactions'] 

cols = ['num_transactions','num_transactions_banded']
df[cols].head()

,num_transactions,num_transactions_banded
0,2,2
1,1,one
2,1,one
3,1,one
4,2,2


[Back to contents](#Home)

## Derive Columns 5 String Manipulation
String manipulations is quite simple

#### Split String Columns<a class="anchor" id="I1"></a>

[Dependency to Create Amount_Banded](#E3)

In [924]:
df['split_left'] = df['Amount_Banded'].str.split ( '-', expand=True).loc [ : , 0 ]
df['split_right'] = df['Amount_Banded'].str.split ('-',expand=True).loc [ : , 1 ]

cols = [ 'Amount_Banded', 'split_left', 'split_right']
df[ cols ].head (7)

,Amount_Banded,split_left,split_right
0,500.01-800,500.01,800
1,1000.01-2000,1000.01,2000
2,1000.01-2000,1000.01,2000
3,500.01-800,500.01,800
4,500.01-800,500.01,800
5,500.01-800,500.01,800
6,800.01-1000,800.01,1000


[Back to contents](#Home)

#### Extract a substring <a class="anchor" id="I2"></a>

In [945]:
# This is a common operation but I cannot think of an example
# Anywyay the 
df ['Amount_Banded'].str.slice ( 0,5 ) [0: 5+1]

0    500.0
1    1000.
2    1000.
3    500.0
4    500.0
5    500.0
Name: Amount_Banded, dtype: object

[Back to contents](#Home)

#### Extract a substring (variable lengths<a class="anchor" id="I2"></a>

In [927]:
# Would like to use Series.str.slice but cannot work out how to pass a dynamic parameter.
# This is exceptionally SLOW!
# Therefore, it is better to re-shape the problem to one that can be solved using str vectorized operations

df['tmp_dash_loc']  = df['Amount_Banded'].str.find ('-')
df ['split_string'] = df.apply (lambda x: x['Amount_Banded'] [ 1+ x['tmp_dash_loc']:], axis=1 )

cols = ['Amount_Banded', 'tmp_dash_loc', 'split_string']
df[cols].head()

,Amount_Banded,tmp_dash_loc,split_string
0,500.01-800,6,800
1,1000.01-2000,7,2000
2,1000.01-2000,7,2000
3,500.01-800,6,800
4,500.01-800,6,800


###### Quicker but more convoluted to use vectorized operations...

In [926]:
# One strategy is to use vectorized operations
# => 
df ['split_string'].str.slice ( 0,10 ) [0: 5+1]

0     800
1    2000
2    2000
3     800
4     800
5     800
Name: split_string, dtype: object

[Back to contents](#Home)

#### Trim a String Column <a class="anchor" id="I3"></a>
(remove blanks at either end)

In [956]:
df ['string with space'] =  ' ' + df['Amount_Banded'].astype('string') + ' '
df ['res1'] = 'start' + df['string with space'].astype(str) + 'end'

df ['string wo space']  = df ['string with space'].str.strip()
df ['res2'] = 'start' +  df['string wo space'].astype(str) + 'end'

cols = ['string with space', 'string wo space', 'res1', 'res2']
df [cols].head(4)


,string with space,string wo space,res1,res2
0,500.01-800,500.01-800,start 500.01-800 end,start500.01-800end
1,1000.01-2000,1000.01-2000,start 1000.01-2000 end,start1000.01-2000end
2,1000.01-2000,1000.01-2000,start 1000.01-2000 end,start1000.01-2000end
3,500.01-800,500.01-800,start 500.01-800 end,start500.01-800end


[Back to contents](#Home)

# Reshaping Data 

#### Prepare tables to pivot

[Dependency to Create Amount_Banded](#E3)

In [26]:
df ['ones'] = 1
df_agg = df.groupby ( ['trans_month', 'cvvVerifyResult']).agg ( 
      num_trans= ('ones', 'count') ,
      sum_trans= ('transactionAmountUSD', sum)
)
df_agg.head(5)

num_trans   sum_trans
trans_month cvvVerifyResult                       
Apr         M                     2288  2322405.00
            N                        6     5799.04
            P                        5     5488.72
            U                        6     7357.90
            X                       14    13076.01

#### Pivot <a class="anchor" id="J1"></a>
See cell above for creating df_agg

In [28]:
df_piv = df_agg.reset_index().pivot( index='trans_month' , columns='cvvVerifyResult', values='num_trans' ).fillna(0)
df_piv.head(5)

cvvVerifyResult,M,N,P,U,X,Y
trans_month,,,,,,
Apr,2288.0,6.0,5.0,6.0,14.0,0.0
Aug,961.0,2.0,2.0,0.0,10.0,0.0
Dec,278.0,1.0,2.0,2.0,2.0,0.0
Feb,4045.0,10.0,20.0,7.0,22.0,2.0
Jan,151.0,1.0,0.0,0.0,2.0,0.0


[Back to contents](#Home)

#### Unpivot <a class="anchor" id="J2"></a>

In [ ]:
df_piv.unstack()

In [ ]:
# An alternative, more flexible aproach but did not work
#df_piv.melt(ignored_index = False )

[Back to contents](#Home)

#### Pivoting 2+ Column Values  <a class="anchor" id="J3"></a>

In [2]:
# Useful function 
# https://stackoverflow.com/questions/14507794/how-to-flatten-a-hierarchical-index-in-columns

def flattenHierarchicalCol(col,sep = '_'):
    if not type(col) is tuple:
        return col
    else:
        new_col = ''
        for leveli,level in enumerate(col):
            if not level == '':
                if not leveli == 0:
                    new_col += sep
                new_col += level
        return new_col

In [30]:
df_piv = df_agg.reset_index().pivot( index='trans_month' , columns='cvvVerifyResult', 
            values= ['num_trans', 'sum_trans']    ).fillna(0)
df_piv.head(5)

num_trans                               sum_trans           \
cvvVerifyResult         M     N     P    U     X    Y           M        N   
trans_month                                                                  
Apr                2288.0   6.0   5.0  6.0  14.0  0.0  2322405.00  5799.04   
Aug                 961.0   2.0   2.0  0.0  10.0  0.0   940532.30  3263.73   
Dec                 278.0   1.0   2.0  2.0   2.0  0.0   282836.18  1616.17   
Feb                4045.0  10.0  20.0  7.0  22.0  2.0  4018406.33  9920.82   
Jan                 151.0   1.0   0.0  0.0   2.0  0.0   144589.31   574.34   

                                                      
cvvVerifyResult         P        U         X       Y  
trans_month                                           
Apr               5488.72  7357.90  13076.01     0.0  
Aug               2041.43     0.00   8634.02     0.0  
Dec               3225.68  1014.57   1362.07     0.0  
Feb              11053.72  6050.92  15746.11  1484.6  
Jan                  0.00     0.00   2144.74     0.0

In [36]:
df_piv.columns = df_piv.columns.map (flattenHierarchicalCol  )
df_piv.head(5)


,num_trans_M,num_trans_N,num_trans_P,num_trans_U,num_trans_X,num_trans_Y,sum_trans_M,sum_trans_N,sum_trans_P,sum_trans_U,sum_trans_X,sum_trans_Y
trans_month,,,,,,,,,,,,
Apr,2288.0,6.0,5.0,6.0,14.0,0.0,2322405.00,5799.04,5488.72,7357.90,13076.01,0.0
Aug,961.0,2.0,2.0,0.0,10.0,0.0,940532.30,3263.73,2041.43,0.00,8634.02,0.0
Dec,278.0,1.0,2.0,2.0,2.0,0.0,282836.18,1616.17,3225.68,1014.57,1362.07,0.0
Feb,4045.0,10.0,20.0,7.0,22.0,2.0,4018406.33,9920.82,11053.72,6050.92,15746.11,1484.6
Jan,151.0,1.0,0.0,0.0,2.0,0.0,144589.31,574.34,0.00,0.00,2144.74,0.0


[Back to contents](#Home)

# Linking Two Data Tables

#### Left Join <a class="anchor" id="K1"></a>
We might want to find the % of each transaction within it's country for a given year.

[Dependency](#G4)

In [73]:
# Create a summary table to join to

df_country_annual_trans_count = df.groupby ( ['ipCountry', 'trans_year'] ).agg ( annual_country_trans_amt = ('transactionAmountUSD','sum')  ).reset_index()

# Rename oe of the columns to make it harder.
df_country_annual_trans_count = df_country_annual_trans_count.rename (columns  =  {'trans_year': 'trans_yr'} )
df_country_annual_trans_count.head(5)

# Note that pandas will add trans_yr to the data frame which seems silly.


,ipCountry,trans_yr,annual_country_trans_amt
0,ad,2007,1.07
1,ae,2007,23965.23
2,af,2007,12651.91
3,al,2007,1595.75
4,ar,2007,41664.91


In [82]:
#annual_country_trans_amt
if  'annual_country_trans_amt' in df.columns:
    df = df.drop ( ['annual_country_trans_amt'] , axis=1)

# Do the left join
df = df.merge ( df_country_annual_trans_count, how='left', \
       left_on=['ipCountry', 'trans_year'], \
       right_on=['ipCountry', 'trans_yr'] ) 

# Calculate a value based on the information now added to the table
df['an_country_amt_perc'] = df.transactionAmountUSD / df.annual_country_trans_amt


# View the results 
cols = ['ipCountry', 'trans_year' , 'transactionAmountUSD', 'annual_country_trans_amt','an_country_amt_perc' ]

# Show the top 2 values (percentages decending) for each country/year group
df.sort_values ( ['ipCountry', 'trans_year', 'an_country_amt_perc'], ascending = [True,True,False] ).\
                    groupby(['ipCountry', 'trans_year']).\
                    head(2).\
                    reset_index(drop=True)\
                    .loc[:,cols].head (8)

,ipCountry,trans_year,transactionAmountUSD,annual_country_trans_amt,an_country_amt_perc
0,ad,2007,1.07,1.07,1.000000
1,ae,2007,1926.21,23965.23,0.080375
2,ae,2007,1905.55,23965.23,0.079513
3,af,2007,1699.26,12651.91,0.134309
4,af,2007,1683.62,12651.91,0.133072
5,al,2007,1595.75,1595.75,1.000000
6,ar,2007,1897.68,41664.91,0.045546
7,ar,2007,1877.39,41664.91,0.045059


[Back to contents](#Home)

#### Union<a class="anchor" id="K2"></a>

In [96]:
cols = ['ipCountry','transactionType', 'transactionCurrencyCode', 'paymentBillingState', 'accountID']

df1 =  df [df['ipCountry']=='gb'][cols] 
df2 =  df [df['ipCountry']=='us'][cols]             

# Do the union
df_res = pd.concat ([df1,df2])
df_res.ipCountry.value_counts()

us    13046
gb     1247
Name: ipCountry, dtype: int64

[Back to contents](#Home)

# Data Cleaning : Null and Blank Values

#### Identify and Replace Nulls <a class="anchor" id="L1"></a>

In [121]:
# Find true number of values (2000)
num_rows = df.shape[0]

# Identify columns with some nulls in them
df_cols_with_nulls = df.count() [df.count() != num_rows].reset_index(drop=False)
df_cols_with_nulls.columns = ['col', 'count']
df_cols_with_nulls.head(6)

,col,count
0,ipState,19996
1,ipPostalCode,19996
2,ipCountry,19996
3,isProxyIP,19772
4,cardType,18178
5,paymentBillingPostalCode,17640


In [142]:
# For a few string columns replace the nulls with 'Not Given' (as an example)
str_cols = list ( df_cols_with_nulls[0:3].col)

# There are ways to automate this but for clarity
df['ipState'] = df['ipState'].fillna ( 'Not Given')
df['ipPostalCode'] = df['ipPostalCode'].fillna ( 'Not Given')
df['ipCountry'] = df['ipCountry'].fillna ( 'Not Given')

df[df.ipState=='Not Given'][ str_cols]



,ipState,ipPostalCode,ipCountry
1666,Not Given,Not Given,Not Given
3338,Not Given,Not Given,Not Given
5413,Not Given,Not Given,Not Given
10823,Not Given,Not Given,Not Given


[Back to contents](#Home)

#### Identify and Replace Blank Strings<a class="anchor" id="L2"></a>
It is likely that read_csv etc is clever enough to turn these to null but just in case

In [150]:
df ['empty_strings'] = ''
# These do not get picked up by isna
df.empty_strings.isna ().value_counts()

False    20000
Name: empty_strings, dtype: int64

In [148]:
# Identify
(df['empty_strings'] == '').sum()

20000

In [153]:
# Set them to null, then the method in the previous section can be used.
boolIndex = df['empty_strings'] == ''
df.loc[boolIndex, 'empty_strings'] = None
df.empty_strings.isna ().value_counts()

True    20000
Name: empty_strings, dtype: int64

[Back to contents](#Home)

#### Identify and Remove White Space Invisible Characters  <a class="anchor" id="L3"></a>
Again the main challenge might be loading the data in the first place. Pandas might handle this quite well

In [160]:
#https://stackoverflow.com/questions/19004080/removing-non-printable-gremlin-chars-from-text-files
# todo, sorry.

[Back to contents](#Home)

# Data Cleaning : Duplicate Values

#### Flag duplicates across a set of columns  <a class="anchor" id="M1"></a>

In [198]:

# There are not true duplicates but 
# assume that there should only be one transction per account per day.

cols_display = ['accountID', 'transactionDate', 'transactionAmountUSD', 'localHour', 'transactionType', 'shippingCountry']

# Use pandas to identify duplicates
cols = ['accountID', 'transactionDate']
df['boolIndexAllDups'] = df.duplicated ( subset = cols, keep= False) # all duplicates

# To remove duplicates using the default pandas framework, use keep='first' (example in next cell)

#cols = ['accountID', 'transactionDate']
#df['boolIndexKeepFirst'] = df.duplicated ( subset = cols, keep= 'first') # only 

cols_display = ['accountID', 'transactionDate', 'transactionAmountUSD', 'localHour', \
                 'transactionType', 'shippingCountry', 'boolIndexAllDups', 'boolIndexKeepFirst']

df.loc[boolIndex, cols_display].sort_values ('accountID').head(5)


,accountID,transactionDate,transactionAmountUSD,localHour,transactionType,shippingCountry,boolIndexAllDups,boolIndexKeepFirst
13359,A2760793813048785,2007-04-22,1892.95,18,P,US,True,True
4675,A2760793813048785,2007-04-22,75.86,18,P,US,True,False
3217,D2747228971238755,2007-02-10,1789.72,2,P,AU,True,False
13195,D2747228971238755,2007-02-10,1720.12,8,P,FI,True,True
17490,D6112963463785322,2007-10-08,680.18,2,P,US,True,True


#### Drop duplicates across a set of columns  <a class="anchor" id="M2"></a>
Pandas offers a drop_duplicates () function. However, I prefer to flag bad rows and then ignore them.
In reality duplicates might be dealt with by ranking them according in order of "best-looking" row and then selecting the 
highest ranked values.
<br>
Another option is to flag the good rows and then just ignore the bad ones in analysis

In [201]:
# The above example did not show this very well because there were no 
# rows duplicated 3+ times

df_test = pd.DataFrame( {'val': ['a', 'a', 'a', 'b', 'c', 'c', 'c'] })
df_test['to_keep'] = ~ df_test.val.duplicated (keep='first')
df_test

,val,to_keep
0,a,True
1,a,False
2,a,False
3,b,True
4,c,True
5,c,False
6,c,False


In [206]:
# So to deduplicate just select the booleanIndex
df_test[df_test['to_keep'] == True ]  ## the == True bit is not needed

,val,to_keep
0,a,True
3,b,True
4,c,True


[Back to contents](#Home)

In [161]:
#### Data Cleaning - Duplicate Values

* [TODO: Check for duplicates in one columns](#M1)
* [TODO: Check for duplicates in multiple columns](#M2)
* [TODO :Flag duplicates for removal](#M3)


SyntaxError: invalid syntax (<ipython-input-161-1b17b8f1a35b>, line 3)

# Scratch

In [ ]:
if not 'Unknown' in df['Amount_Banded'].cat.categories:
   df['Amount_Banded'] = df['Amount_Banded'].cat.add_categories ( 'Unknown')
df['ColToSplit'] = df['Amount_Banded'].fillna('Unknown', inplace=False) + df.transactionCurrencyCode


In [875]:
import pandas as pd
df = pd.DataFrame  ( {'st_col1':['aa-b', 'aaa-b']}  )
df['index_dash'] = df['st_col1'].str.find ('-')

# gives wrong answer at index 1
df['res_wrong'] =   df['st_col1'].str.slice (3)

# what I want to do :
df['res_cant_do'] = df['st_col1'].str.slice ( df['index_dash']  )

# Slow solution
df['slow_sol'] = df.apply (lambda x: x['st_col1'] [ 1+ x['index_dash']:], axis=1 )

df['st_col']= ['aab', 'aaab'] 



df

,st_col1,index_dash,res_wrong,res_cant_do,slow_sol,st_col
0,aa-b,2,b,NaN,b,aab
1,aaa-b,3,-b,NaN,b,aaab
